#**Chargement des packages et datas**

In [0]:
import pandas as pd
from google.colab import drive 
drive.mount('/gdrive', force_remount=True)
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
df_train = pd.read_csv('/gdrive/My Drive/Kaggle/liverpool-ion-switching/train.csv')
df_test = pd.read_csv('/gdrive/My Drive/Kaggle/liverpool-ion-switching/test.csv')
y_train_proba = np.load(f"/gdrive/My Drive/Kaggle/liverpool-ion-switching/Y_train_proba.npy")
y_test_proba = np.load(f"/gdrive/My Drive/Kaggle/liverpool-ion-switching/Y_test_proba.npy")

#**Mise en forme des datas**

In [0]:
def create_feature (proba, df):
  proba_label_0=[]
  for i in range(proba.shape[0]):
    proba_label_0.append(proba[i][0])
  df['proba_label_0'] = proba_label_0

  proba_label_1=[]
  for i in range(proba.shape[0]):
    proba_label_1.append(proba[i][1])
  df['proba_label_1'] = proba_label_1

  proba_label_2=[]
  for i in range(proba.shape[0]):
    proba_label_2.append(proba[i][2])
  df['proba_label_2'] = proba_label_2

  proba_label_2=[]
  for i in range(proba.shape[0]):
    proba_label_2.append(proba[i][2])
  df['proba_label_2'] = proba_label_2

  proba_label_3=[]
  for i in range(proba.shape[0]):
    proba_label_3.append(proba[i][3])
  df['proba_label_3'] = proba_label_3

  proba_label_4=[]
  for i in range(proba.shape[0]):
    proba_label_4.append(proba[i][4])
  df['proba_label_4'] = proba_label_4

  proba_label_5=[]
  for i in range(proba.shape[0]):
    proba_label_5.append(proba[i][5])
  df['proba_label_5'] = proba_label_5

  proba_label_6=[]
  for i in range(proba.shape[0]):
    proba_label_6.append(proba[i][6])
  df['proba_label_6'] = proba_label_6

  proba_label_7=[]
  for i in range(proba.shape[0]):
    proba_label_7.append(proba[i][7])
  df['proba_label_7'] = proba_label_7

  proba_label_8=[]
  for i in range(proba.shape[0]):
    proba_label_8.append(proba[i][8])
  df['proba_label_8'] = proba_label_8

  proba_label_9=[]
  for i in range(proba.shape[0]):
    proba_label_9.append(proba[i][9])
  df['proba_label_9'] = proba_label_9

  proba_label_10=[]
  for i in range(proba.shape[0]):
    proba_label_10.append(proba[i][10])
  df['proba_label_10'] = proba_label_10

In [0]:
create_feature(y_train_proba, df_train)
create_feature(y_test_proba, df_test)

In [0]:
df = df_train.copy() # Va contenir le training set préparé proprement
df['signal'].at[478587] = df_train['signal'].iloc[0:478586].mean() # Retire les outliers du sample numero 1
df['signal'].at[478609] = df_train['signal'].iloc[0:478609].mean()
df['signal'].at[478610] = df_train['signal'].iloc[0:478610].mean()

In [0]:
import numpy.polynomial.polynomial as poly 
n_sample = 500000
degree = 2 # Correction de la rampe du sample numero 2
t=df_train['time'].iloc[500000:500000+100000]
f=df_train['signal'].iloc[500000:500000+100000]
coefs = poly.polyfit(t, f, degree)
ffit = poly.polyval(t, coefs)
df.loc[500000:500000+100000-1, 'signal'] = f-ffit + df_train['signal'].iloc[500000+100000-1:500000*2].mean()

In [0]:
#Correction des rampes du test set
df_test.loc[100000:200000-1,'signal'] = df_test['signal'].iloc[100000:200000].values - ffit.values + df_train['signal'].iloc[500000+100000-1:500000*2].mean()
df_test.loc[700000:800000-1,'signal'] = df_test['signal'].iloc[700000:800000].values - ffit.values + df_train['signal'].iloc[500000+100000-1:500000*2].mean()
df_test.loc[600000:700000-1,'signal'] = df_test['signal'].iloc[600000:700000].values - ffit.values + df_train['signal'].iloc[500000+100000-1:500000*2].mean()
df_test.loc[0:100000-1,'signal'] = df_test['signal'].iloc[0:100000].values - ffit.values + df_train['signal'].iloc[500000+100000-1:500000*2].mean()
df_test.loc[400000:500000-1,'signal'] = df_test['signal'].iloc[400000:500000].values - ffit.values + df_train['signal'].iloc[500000+100000-1:500000*2].mean()
df_test.loc[800000:900000-1,'signal'] = df_test['signal'].iloc[800000:900000].values - ffit.values + df_train['signal'].iloc[500000+100000-1:500000*2].mean()

In [0]:
degree = 10 # Correction de toutes les paraboles dans le training et test set
n_batch = 6
f = df_train['signal'].iloc[n_sample*n_batch:n_sample*(n_batch+1)]
t = df_train['time'].iloc[n_sample*n_batch:n_sample*(n_batch+1)]
coefs = poly.polyfit(t, f, degree)
ffit = poly.polyval(t, coefs)
for n_batch in [6,7,8,9]: 
  df.loc[n_sample*n_batch:n_sample*(n_batch+1)-1,'signal'] = df.loc[n_sample*n_batch:n_sample*(n_batch+1)-1,'signal'].values-ffit.values
df_test['signal'].iloc[2*n_sample:3*n_sample] = f.values-ffit.values

/usr/local/lib/python3.6/dist-packages/numpy/polynomial/polynomial.py:1336: RankWarning: The fit may be poorly conditioned
  return pu._fit(polyvander, x, y, deg, rcond, full, w)


In [0]:
ref = min(df['signal'].iloc[n_sample*0:n_sample*(0+1)].loc[(df.open_channels == 1)])
for n_batch in range(10): # Correction des offsets de tous batchs
  if n_batch == 8:
    f = df['signal'].iloc[n_sample*n_batch:n_sample*(n_batch+1)]
    label_1_mean_pos = df['signal'].iloc[n_sample*n_batch:n_sample*(n_batch+1)].loc[(df.open_channels == 1)] # Posision moyenne des label égale à 1, seule classe commune à tous les batch du train set
    df.loc[n_sample*n_batch:n_sample*(n_batch+1)-1,'signal'] = f-label_1_mean_pos.mean()
    df_test.loc[1000000:1500000-1,'signal'] = df_test.loc[1000000:1500000-1,'signal'].values-label_1_mean_pos.mean()
  elif n_batch == 0:
    f = df['signal'].iloc[n_sample*n_batch:n_sample*(n_batch+1)]
    label_1_mean_pos = df['signal'].iloc[n_sample*n_batch:n_sample*(n_batch+1)].loc[(df.open_channels == 1)] # Posision moyenne des label égale à 1, seule classe commune à tous les batch du train set
    df.loc[n_sample*n_batch:n_sample*(n_batch+1)-1,'signal'] = f-label_1_mean_pos.mean()
    df_test.loc[0:500000-1,'signal'] = df_test.loc[0:500000-1,'signal'].values-label_1_mean_pos.mean()
    df_test.loc[600000:700000-1,'signal'] = df_test.loc[600000:700000-1,'signal'].values-label_1_mean_pos.mean()
    df_test.loc[800000:1000000-1,'signal'] = df_test.loc[800000:1000000-1,'signal'].values-label_1_mean_pos.mean()
    df_test.loc[1500000:20000000-1,'signal'] = df_test.loc[1500000:20000000-1,'signal'].values-label_1_mean_pos.mean() 
  elif n_batch == 4:
    f = df['signal'].iloc[n_sample*n_batch:n_sample*(n_batch+1)]
    label_1_mean_pos = df['signal'].iloc[n_sample*n_batch:n_sample*(n_batch+1)].loc[(df.open_channels == 1)] # Posision moyenne des label égale à 1, seule classe commune à tous les batch du train set
    df.loc[n_sample*n_batch:n_sample*(n_batch+1)-1,'signal'] = f-label_1_mean_pos.mean()+0.05
    df_test.loc[500000:600000-1,'signal'] = df_test.loc[500000:600000-1,'signal'].values-label_1_mean_pos.mean()
    df_test.loc[700000:800000-1,'signal'] = df_test.loc[700000:800000-1,'signal'].values-label_1_mean_pos.mean()
  elif n_batch == 9:
    f = df['signal'].iloc[n_sample*n_batch:n_sample*(n_batch+1)]
    label_1_mean_pos = df['signal'].iloc[n_sample*n_batch:n_sample*(n_batch+1)].loc[(df.open_channels == 1)] # Posision moyenne des label égale à 1, seule classe commune à tous les batch du train set
    df.loc[n_sample*n_batch:n_sample*(n_batch+1)-1,'signal'] = f-label_1_mean_pos.mean()+0.28
  else:
    f = df['signal'].iloc[n_sample*n_batch:n_sample*(n_batch+1)]
    label_1_mean_pos = df['signal'].iloc[n_sample*n_batch:n_sample*(n_batch+1)].loc[(df.open_channels == 1)] # Posision moyenne des label égale à 1, seule classe commune à tous les batch du train set
    df.loc[n_sample*n_batch:n_sample*(n_batch+1)-1,'signal'] = f-label_1_mean_pos.mean()

#**Machine learning**

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
list_batch_7 = list(df['signal'].iloc[7*n_sample+125000:8*n_sample-125000].index)
X_train, X_val, y_train, y_val = train_test_split(df.drop(list_batch_7,axis=0,inplace=False).drop(['open_channels', 'time'], axis=1).values, 
                                                  df.drop(list_batch_7,axis=0,inplace=False)['open_channels'].values, test_size=0.2,
                                                  random_state=0)

In [0]:
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

XGB = XGBClassifier(n_estimators=10, verbosity = 1, n_jobs=-1, max_depth=5)

DT = DecisionTreeClassifier(max_depth = 12)

knn = KNeighborsClassifier(n_neighbors=40)

In [0]:
from mlxtend.classifier import StackingClassifier

stack_gen = StackingClassifier(classifiers=(knn, DT),
                                meta_classifier=XGB,
                                use_features_in_secondary=True, verbose = 3)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [0]:
stack_gen_model = stack_gen.fit(np.array(X_train), np.array(y_train))

Fitting 2 classifiers...
Fitting classifier1: kneighborsclassifier (1/2)
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=40, p=2,
                     weights='uniform')
Fitting classifier2: decisiontreeclassifier (2/2)
DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=12, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')


In [0]:
from sklearn.metrics import f1_score

y_pred = stack_gen_model.predict(np.array(X_train))
print('f1_score_train:', f1_score(y_train, y_pred, average="macro"))
y_pred = stack_gen_model.predict(np.array(X_val))
print('f1_score_val:', f1_score(y_val, y_pred, average="macro"))

f1_score_train: 0.9405946764285897
f1_score_val: 0.9387309385801713


In [0]:
X_test = df_test.drop(['time'],axis=1).values

In [0]:
pred_ = stack_gen_model.predict(np.array(X_test))

In [0]:
df_submission = pd.read_csv('/gdrive/My Drive/Kaggle/liverpool-ion-switching/sample_submission.csv') # Ouverture du template de submission

In [0]:
df_submission['open_channels'] = pred_ # Remplacement de la collone des predictions

In [0]:
df_submission.to_csv(r'/gdrive/My Drive/Kaggle/liverpool-ion-switching/Submission/submission_30-stack_XGBoost_with_new_features.csv', index = False, float_format='%.4f')